In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [2]:
url = 'https://economia.uol.com.br/cotacoes/'

In [3]:
def ProdutosAgricolas():
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    # Encontra div das 3 últimas tabelas do site, Inflação, Commodities e Produtos agrícolas
    divs = soup.findAll('div', attrs={'class':'col-lg-6 col-md-6 col-sm-12 col-xs-12 accordion'})

    res = []
    table = []

    # Para cada div, extrai o que tem na table do html (retorna as 3 tabelas)
    for div in divs:
        table.extend(div.find('table'))

    # Pega apenas a table[5], com os dados dos Produtos agrícolas e separa por tr do html 
    table_rows = table[5].find_all('tr')

    for tr in table_rows: # Para cada linha
        date = table[4].text # Extrai a o texto da table[4], com a última atualização
        td = tr.find_all('td') # Separa em listas cada conteúdo (td) da linha do html 
        row = [tr.text for tr in td] 
        row = row + [date] 
        if row:
            res.append(row)
    
    # Tenta ler a tabela já criada, se ainda não existe, cria uma nova
    try:
        old = pd.read_excel('DadosAgricolas.xlsx') 
    except Exception:
        print("Arquivo 'DadosAgricolas' ainda não criado, gerando novo...")
        vazio = []
        vazio = pd.DataFrame(vazio)
        vazio.to_excel('DadosAgricolas.xlsx', index = False)
        old = pd.read_excel('DadosAgricolas.xlsx')
    
    df = pd.DataFrame(res, columns=['Produto', 'Preco', 'Data'])
    df = df.dropna()
    df = df.append(old) # Append das novas linhas com as antigas, do Excel já criado
    df.drop_duplicates(subset=['Produto', 'Data'], keep='first', inplace=True)
    df.to_excel('DadosAgricolas.xlsx', index = False)


In [4]:
ProdutosAgricolas()
pd.read_excel('DadosAgricolas.xlsx')

,Produto,Preco,Data
0,Açúcar,"R$ 153,03 por saca de 50kg",20 DE DEZEMBRO DE 2021
1,Algodão,"R$ 6,4095 por libra-peso",20 DE DEZEMBRO DE 2021
2,Arroz,"R$ 62,52 por saca de 50kg",20 DE DEZEMBRO DE 2021
3,Bezerro,"R$ 3.035,02 por cabeça ou uni.",20 DE DEZEMBRO DE 2021
4,Boi Gordo,"R$ 317,70 por arroba",20 DE DEZEMBRO DE 2021
5,Café,"R$ 1.432,67 por saca de 60kg",20 DE DEZEMBRO DE 2021
6,Milho,"R$ 87,57 por saca de 60kg",20 DE DEZEMBRO DE 2021
7,Açúcar,"R$ 154,46 por saca de 50kg",17 DE DEZEMBRO DE 2021
8,Algodão,"R$ 6,4036 por libra-peso",17 DE DEZEMBRO DE 2021
9,Arroz,"R$ 62,52 por saca de 50kg",17 DE DEZEMBRO DE 2021


In [ ]:
# while True:
#     ProdutosAgricolas()
#     time.sleep(86400)